In [190]:
# author Chong Teng
# experimental pKa data from DataWarrior http://www.openmolecules.org/index.html

import os
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np

In [173]:
# Smiles to mol type
def smile_to_mol(smile,name="None"):            
    try:

        m = Chem.MolFromSmiles(smile)
        m2=Chem.AddHs(m)
        m2.SetProp("_Name",name)
        AllChem.EmbedMolecule(m2,randomSeed=0xf00d)
    except:
        m2=None
    return m2

In [174]:
#m=smile_to_mol("CCSC([C@H](C1)N(C)[C@@H]2CC1=O)[C@H]2O") #test

In [175]:
df = pd.read_csv(r"./pKaInWater.dwar",sep="	")

In [176]:
print(df["Smiles"])

0             Fc(cc1)ccc1S
1            Sc1cccc(Cl)c1
2            Sc(cc1)ccc1Cl
3            Sc(cc1)ccc1Br
4               SCc1ccccc1
               ...        
7908             Nc1nccnc1
7909             Nc1ncncc1
7910             Nc1ncncc1
7911    Nc1[nH+]c(O)c[nH]1
7912     Cc1nc([O-])n[nH]1
Name: Smiles, Length: 7913, dtype: object


In [177]:
df['m']=df['Smiles'].apply(smile_to_mol)
df.dropna(inplace=True)
df.index=ExcelRead['Row-ID']

In [178]:
df[df["basicOrAcidic"]=="acidic"]


,colorInfo,FragFp,OrgFunctions,Structure,pKa,temp,method,type,basicOrAcidic,Row-ID,Smiles,m
Row-ID,,,,,,,,,,,,
1,red:0,0F8F#1)F$8/#8///08!,h2Sh0AVQSV,daD@P@BNdDfUzZ@B@@,6.35,32,mean from 2,a1/apparent,acidic,1,Fc(cc1)ccc1S,<rdkit.Chem.rdchem.Mol object at 0x0000025DC73...
2,red:6,"0F8F#1.80H&2)8/,2/&",h2Sh8AVQSV,daD@P@bFBDfUjz@H@@,5.76,25,mean from 2,a1/apparent,acidic,2,Sc1cccc(Cl)c1,<rdkit.Chem.rdchem.Mol object at 0x0000025DC73...
3,red:7,"0F8F#1)F$8)2)8/,2/&",h2Sh8AVQSV,daD@P@bNBDfUzZ@B@@,5.82,25,mean from 2,a1/apparent,acidic,3,Sc(cc1)ccc1Cl,<rdkit.Chem.rdchem.Mol object at 0x0000025DBB2...
5,red:0,kFFF$8!F%F#8/#8///$,h2SplAYvSV,daD@`@BDeeVz`@@@,9.43,25,spectrophotometric,a1/apparent,acidic,5,SCc1ccccc1,<rdkit.Chem.rdchem.Mol object at 0x0000025DC73...
6,red:0,FF8F#18(F$8)1)8///$,_2ShkCik,daD@`@BDfUzZ@B@@,6.47,25,mean from 2,a1/apparent,acidic,6,Cc(cc1)ccc1S,<rdkit.Chem.rdchem.Mol object at 0x0000025DC73...
...,...,...,...,...,...,...,...,...,...,...,...,...
7897,red:3,fpQW20O4!4N#8B80V#V!F0804042F01&8410V0X!8!40F(...,C4M2ME838nEX,gOz@AEjeKnKST@@,6.06,25,potentiometric,a/apparent,acidic,7897,CNc1nnn[nH]1,<rdkit.Chem.rdchem.Mol object at 0x0000025DC74...
7898,red:3,vpQW21N4!4N0F08D8%V!F0804!2F01&8410V0X!8!40F!V...,C4M2ME238m,gOzAAEfyRewEij@@,2.62,25,potentiometric,a1/apparent,acidic,7898,[NH3+]Cc1nnn[nH]1,<rdkit.Chem.rdchem.Mol object at 0x0000025DC74...
7907,red:4,0F@F#1.8//(F/%2#1!VF,C4M4Sf83EP,gO|@AbeLmXD@@@,20.50,24.9,spectrophotometric,a1/apparent,acidic,7907,Nc1ncccn1,<rdkit.Chem.rdchem.Mol object at 0x0000025DC74...


In [179]:
AllChem.CalcNumRings(df['m'][351])

0

In [185]:
m3=AllChem.GetMorganFingerprintAsBitVect(m,2,nBits=1024) # TODO   Convert  Morgan Fingerprints